In [ ]:
%run requirements.ipynb
%run config.ipynb

In [ ]:
class FdiGenerator:
    def __init__(self):
        self.alpha_max = 0.8
        self.alpha_min = 0.2
        self.t2 = 0
        self.t1 = 0
    
    # x is the origin value

    def fdi_1(self, x):
#         print(f'-------fdi1:')
        fdi =  x * self.get_alpha()
        return fdi

    def fdi_2(self, x, df_max):
#         print(f'-------fdi2:')
        alpha = self.get_alpha()
        gamma = alpha * df_max
        return gamma if (x > gamma) else x

    def fdi_3(self, x, df_max):
#         print(f'-------fdi3:')
        gamma = np.random.uniform(0, 1) * df_max
        return max((x - gamma), 0)
                
    def fdi_4(self, x, index, total_index):
#         print(f'-------fdi4: x={x}, t1={self.t1}, t2={self.t2}')
        if index > self.t1 and index < self.t2:
#             print('-----------return 0')
            return 0
        return x
    
    def fdi_5(self, x, index):
#         print(f'-------fdi5:')
        alpha = self.get_alpha()
        return x * alpha
    
    def fdi_6(self, avg_x):
#         print(f'-------fdi6:')
        alpha = self.get_alpha()
        return avg_x * alpha
    
    def get_alpha(self):
        return np.random.uniform(self.alpha_min, self.alpha_max)
    
    def get_random_mode(self):
        # return an integer in range from 1 to 6
        fdi_method = np.random.randint(1, 7)
        self.t1 = np.random.randint(0, 20)
        self.t2 = np.random.randint(self.t1+5, 25)
        
        return fdi_method
    
    def random_fdi(self, fdi_method, x, df, index):
        # return a fdi value with fdi_method provided
        if fdi_method == 1:
            return self.fdi_1(x)
        elif fdi_method == 2:
            return self.fdi_2(x, df.max())
        elif fdi_method == 3:
            return self.fdi_3(x, df.max())
        elif fdi_method == 4:
            return self.fdi_4(x, index, df.shape[0])
        elif fdi_method == 5:
            return self.fdi_5(x, index)
        else:
            return self.fdi_6(df.mean())

In [ ]:
# split data into groups
def split_data_to_group(target_df, areas, METER_IN_RANGE, NUMBER_OF_HALF_HOURS, TOTAL_METER):
    data_group = {}
    for group in range(areas):
        if group == (AREA_NUM - 1):
            data_group[group] = target_df[METER_IN_RANGE*group*(NUMBER_OF_HALF_HOURS) : METER_IN_RANGE*(group+1)*(NUMBER_OF_HALF_HOURS)+NUMBER_OF_HALF_HOURS* (TOTAL_METER % areas)]
        else:
            data_group[group] = target_df[METER_IN_RANGE*group*(NUMBER_OF_HALF_HOURS) : METER_IN_RANGE*(group+1)*(NUMBER_OF_HALF_HOURS)]
    return data_group

In [ ]:
def fdi_attack(group, total_meter, area_num, group_meter, total_day, ntl_meter_ratio, ntl_day_ratio):
    # ex: split_group(data_group, total_meter = 2044, group_meter = 204, total_day = 299, ntl_meter_ratio = 0.1, ntl_day_ratio = 0.5)
    
    number_of_half_hours = total_day * 48
    
    ntl_num = round(group_meter * ntl_meter_ratio) # ntl_num = how many number of ntl meters
    ntl_day_num = round(total_day * ntl_day_ratio) # ntl_day_num = how many days are fdi for ntl meters
    
    np.random.seed(42)
    
    final_data_group = []
    origin_data_group = []
    
    print('start iterate 10 area')
    # iterate through 10 area
    for area, data in group.items():
        data = data.copy(deep=True)
        num = int(data.shape[0] / 530 / 48)
        data['FDI'] = False
        data['Atk_Elec'] = data['Electricity']
        data['FDI_method'] = 0
#         origin_data = data.copy(deep=True)
        
        print('start split meter in area ', area)
        #split total meter into normal meters and fdi meters
        rs = ShuffleSplit(n_splits=1, test_size=ntl_num, random_state=42)
        for normal_index, fdi_index in rs.split(range(num)):
            normal_index = normal_index
            fdi_index = fdi_index
#             print('len of fdi_index', len(fdi_index))
            print(fdi_index)
            
#         print('start iterate fdi meter')
        # iterate every fdi meter to generate data
        for index in tq.tqdm(fdi_index): 
#             print('index: ',index)
            #index = dataframe day offset
            
            # split total day into normal and fdi day with the ntl_day_num
            rs = ShuffleSplit(n_splits=1, test_size=ntl_day_num, random_state=42)
            for normal_day, fdi_day in rs.split(range(total_day)):
                normal_day = normal_day
                fdi_day = fdi_day
            
#             print('fdi da??y = ', fdi_day)
            baseoffset = index * 530 * 48
            
#             print('start process fdi day')
            # process FDI in fdi days
            for day in fdi_day:
                dayoffset = baseoffset + day*48
                for hour in range(0, 48):
#                     prisnt(f'\nstarting info: index = {index}, day = {fdi_day[day]}, dayoffset = {dayoffset}, hour={hour}')        
                    # one day has 48 data, so every 48 will change a fdi mode
                    if hour % 48 == 0:
                        fdi_method = FDI.get_random_mode()
#                         print('method: ',fdi_method)
                    
                    origin_value = data["Atk_Elec"].iloc[dayoffset + hour]
                    data["Atk_Elec"].iloc[dayoffset + hour] = FDI.random_fdi(fdi_method, origin_value, data["Atk_Elec"][dayoffset:dayoffset+48], hour)
                    data["FDI"].iloc[dayoffset + hour] = True
                    data["FDI_method"].iloc[dayoffset + hour] = fdi_method
#                     print(f'hour={hour}. before: {origin_value}, after: {data["Electricity"].iloc[dayoffset+hour]}')
        assert 1 == 2
        final_data_group.append(data) 
#         origin_data_group.append(origin_data)
        
#     return final_data_group, origin_data_group
    return final_data_group

In [ ]:
def plot_data(normal_data, fdi_data, title, label1, label2, plot_house=5, plot_hour=120, save=False):
    #plotting the points  
    plt.figure(dpi=150, figsize=(15,4))
    if normal_data is not None:
        count = 0
    for i in range(len(normal_data)):
        data = normal_data[i]["Electricity"]
        count += 1
        if count>plot_house:
            break
        plt.plot(HOURS_LIST[:plot_hour], data[:plot_hour], color='blue') 

    if fdi_data is not None:
        count = 0
    for i in range(len(fdi_data)):
        data = fdi_data[i]["Electricity"]
        count += 1
        if count>plot_house:
            break
        plt.plot(HOURS_LIST[:plot_hour], data[:plot_hour], color='red')
    
    
    blue_patch = mpatches.Patch(color='blue', label=label1)
    red_patch = mpatches.Patch(color='red', label=label2)
    plt.legend(handles=[blue_patch, red_patch])       
        
    # naming the x axis 
    plt.xlabel('Half Hours') 
    # naming the y axis 
    plt.ylabel('Electricity Consumption(W)') 

    # giving a title to my graph 
    plt.title(title) 

    # function to save the plot
    if save == True:
        save_fig(title)

    # function to show the plot 
    plt.show()